# Assistants API 사용 방법 알아보기

새로운 [Assistants API](https://platform.openai.com/docs/assistants/overview)는 [Chat Completions API](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)의 발전된 형태로, 보다 간단하게 assistant를 만들고, 개발자가 Code Interpreter 및 Retrieval과 같은 강력한 도구에 접근할 수 있도록 하는 것을 목적으로 합니다.

![Assistants API Diagram](images/assistants_overview_diagram.png)

## Chat Completions API vs Assistants API

**Chat Completions API**의 기본 단위는 `Messages`이며, 여기에 `Model`(`gpt-3.5-turbo`, `gpt-4` 등)을 사용하여 `Completion`을 수행합니다. 이 API는 가볍고 강력하지만 본질적으로 상태가 없기 때문에 대화 상태, 도구 정의, 검색 문서, 코드 실행을 수동으로 관리해야 합니다.

**Assistants API**의 기본 단위는 다음과 같습니다:

- 기본 모델, 지침, 도구, 문서(문맥)를 포함하는 `Assistants`,
- 대화의 상태를 나타내는 `Threads`,
- 텍스트 응답 및 다단계 도구 사용을 포함하여 `Thread`에서 `Assistant`의 실행을 구동하는 `Runs`.

이러한 기능들을 사용하여 강력하고 상태가 있는 경험을 만드는 방법을 살펴보겠습니다.

## Setup

### Python SDK

In [9]:
!pip install --upgrade openai

In [10]:
!pip show openai | findstr Version

Version: 1.51.2


### Pretty Printing Helper


In [11]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## Complete Example with Assistants API


### Assistants


Assistants API를 시작하는 가장 쉬운 방법은 [Assistants Playground](https://platform.openai.com/playground)를 통하는 것입니다.

![Assistants Playground](images/assistants_overview_assistants_playground.png)


assistant를 만들어 보겠습니다! [문서](https://platform.openai.com/docs/assistants/overview)에 있는 것처럼 수학 과외 선생님을 만들어 보겠습니다.

![Creating New Assistant](images/assistants_overview_new_assistant.png)


생성한 assistant들은 [Assistants Dashboard](https://platform.openai.com/assistants)에서 볼 수 있습니다.

![Assistants Dashboard](images/assistants_overview_assistants_dashboard.png)


또한 다음과 같이 Assistants API를 직접 사용하여 Assistant를 생성할 수도 있습니다:

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# .env 파일에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("API 키가 설정되지 않았습니다. 환경 변수를 확인하세요.")
else:
    print("API 키가 성공적으로 불러와졌습니다.")

API 키가 성공적으로 불러와졌습니다.


In [13]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘",
    model="gpt-4o",
)
show_json(assistant)

{'id': 'asst_NumaNLIgmpjeAKo16jh0PKPJ',
 'created_at': 1728579022,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None, 'file_search': None},
 'top_p': 1.0}

대시보드를 통해 Assistant를 생성하든 API를 사용하든, Assistant ID를 추적하는 것이 중요합니다. 이 ID를 통해 Threads와 Runs 전반에 걸쳐 Assistant를 참조하게 됩니다.


다음으로, 새로운 Thread를 생성하고 그 안에 Message를 추가하겠습니다. 이렇게 하면 대화의 상태를 유지할 수 있어, 매번 전체 메시지 기록을 다시 보낼 필요가 없습니다.

### Threads


새로운 Thread 생성하기

In [9]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_KfZbQYNHHNcLc0lazOirJ3zn',
 'created_at': 1728575837,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

thread에 message 추가하기

In [10]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="방정식 '3x + 11 = 14'를 풀어줘"
)
show_json(message)

{'id': 'msg_0EPqVBvffWMtB3DVzfsjJJVe',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [], 'value': "방정식 '3x + 11 = 14'를 풀어줘"},
   'type': 'text'}],
 'created_at': 1728575854,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_KfZbQYNHHNcLc0lazOirJ3zn'}

> **Note**
> 매번 전체 기록을 보내지 않더라도, 각 Run마다 전체 대화 기록의 토큰에 대해 요금이 청구됩니다.


### Runs

우리가 만든 Thread가 앞서 만든 Assistant와 **연결되지 않았음**을 주목하세요! Thread는 Assistants와 독립적으로 존재하며, 이는 ChatGPT(모델/GPT에 thread가 연결되어 있는)를 사용해 본 사람들이 예상하는 것과 다를 수 있습니다.

주어진 Thread에 대한 Assistant의 Completion을 얻으려면 Run을 생성해야 합니다. Run을 생성하면 Assistant에게 Thread의 메시지를 살펴보고 조치를 취하라는 지시가 됩니다. 단일 응답을 추가하거나 도구를 사용할 수 있습니다.

> **참고**
> Runs는 Assistants API와 Chat Completions API 사이의 주요 차이점입니다. Chat Completions에서는 모델이 단일 메시지로만 응답할 수 있는 반면, Assistants API에서는 Run을 통해 Assistant가 하나 또는 여러 도구를 사용하고 Thread에 여러 메시지를 추가할 수 있습니다.

우리 Assistant에게 사용자에게 응답하도록 하려면 Run을 생성합시다. 앞서 언급했듯이, Assistant와 Thread를 _모두_ 지정해야 합니다.

In [11]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_HR1zNgIixNM3T3h7MX4QRm3a',
 'assistant_id': 'asst_dL7GXryfiTzV7zAQ1rvE72xI',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1728575887,
 'expires_at': 1728576487,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_KfZbQYNHHNcLc0lazOirJ3zn',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': None,
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

Chat Completions API에서 완성을 생성하는 것과 달리, **Run을 생성하는 것은 비동기 작업입니다**. 이 작업은 Run의 메타데이터와 함께 즉시 반환되며, 여기에는 처음에 `queued`로 설정된 `status`가 포함됩니다. Assistant가 도구 사용과 메시지 추가와 같은 작업을 수행함에 따라 `status`가 업데이트됩니다.

Assistant가 처리를 완료했는지 알기 위해, 우리는 Run을 반복적으로 폴링할 수 있습니다. (스트리밍 지원이 곧 제공될 예정입니다!) 여기서는 `queued` 또는 `in_progress` 상태만 확인하지만, 실제로 Run은 사용자에게 표시할 수 있는 [다양한 상태 변경](https://platform.openai.com/docs/api-reference/runs/object#runs/object-status)을 겪을 수 있습니다. (이것들을 Steps라고 하며 나중에 다룰 것입니다.)

In [12]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [13]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_HR1zNgIixNM3T3h7MX4QRm3a',
 'assistant_id': 'asst_dL7GXryfiTzV7zAQ1rvE72xI',
 'cancelled_at': None,
 'completed_at': 1728575888,
 'created_at': 1728575887,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1728575887,
 'status': 'completed',
 'thread_id': 'thread_KfZbQYNHHNcLc0lazOirJ3zn',
 'tool_choice': 'auto',
 'tools': [],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 8, 'prompt_tokens': 68, 'total_tokens': 76},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

### Messages


Run이 완료되었으므로, Assistant가 무엇을 추가했는지 보기 위해 Thread 안의 메시지들을 나열할 수 있습니다.

In [14]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_OdqrGcLYVjuoot4pvQvPjJrd',
   'assistant_id': 'asst_dL7GXryfiTzV7zAQ1rvE72xI',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [], 'value': 'x = 1입니다.'},
     'type': 'text'}],
   'created_at': 1728575888,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_HR1zNgIixNM3T3h7MX4QRm3a',
   'status': None,
   'thread_id': 'thread_KfZbQYNHHNcLc0lazOirJ3zn'},
  {'id': 'msg_0EPqVBvffWMtB3DVzfsjJJVe',
   'assistant_id': None,
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "방정식 '3x + 11 = 14'를 풀어줘"},
     'type': 'text'}],
   'created_at': 1728575854,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'status': None,
   'thread_id': 'thread_KfZbQYNHHNcLc0lazOirJ3zn'}],
 'object': 'list'

보시다시피 메시지들은 역시간 순서로 정렬되어 있습니다 - 이렇게 함으로써 가장 최근의 결과가 항상 첫 번째 '페이지'에 있게 됩니다(결과는 페이지네이션 될 수 있으므로). 이는 Chat Completions API의 메시지 순서와 반대이므로 주의해야 합니다.

우리의 Assistant에게 결과에 대해 좀 더 설명해달라고 요청해 보겠습니다!

In [15]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="설명해 주시겠어요?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_nyfGN41q9uKdYRFUeEQNjHYY',
   'assistant_id': 'asst_dL7GXryfiTzV7zAQ1rvE72xI',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '양변에서 11을 빼면, \\(3x = 3\\). 양변을 3으로 나누면, \\(x = 1\\)입니다.'},
     'type': 'text'}],
   'created_at': 1728576011,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_mrL2uhgGRdtfoJF7vVSwehu8',
   'status': None,
   'thread_id': 'thread_KfZbQYNHHNcLc0lazOirJ3zn'}],
 'object': 'list',
 'first_id': 'msg_nyfGN41q9uKdYRFUeEQNjHYY',
 'last_id': 'msg_nyfGN41q9uKdYRFUeEQNjHYY',
 'has_more': False}

이것이 간단한 예제를 위해 응답을 받기까지 많은 단계가 필요한 것처럼 느껴질 수 있습니다. 그러나, 우리는 코드를 거의 변경하지 않고도 우리의 Assistant에 매우 강력한 기능을 추가할 수 있는 방법을 곧 보게 될 것입니다!

### Example


이 모든 것을 어떻게 결합할 수 있는지 살펴보겠습니다. 아래는 생성한 Assistant를 사용하기 위해 필요한 모든 코드입니다.

우리가 이미 수학 Assistant를 만들었기 때문에, 그 ID를 `MATH_ASSISTANT_ID`에 저장했습니다. 그런 다음 두 개의 함수를 정의했습니다:

- `submit_message`: Thread에 Message를 생성한 다음, 새로운 Run을 시작하고 반환합니다.
- `get_response`: Thread의 메시지 목록을 반환합니다.

In [16]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI()

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

또한 재사용할 수 있는 `create_thread_and_run` 함수를 정의했는데, 이는 실제로 우리 API의 [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) 복합 함수와 거의 동일합니다. 마지막으로, 우리는 각각의 가상 사용자 요청을 새로운 Thread에 제출할 수 있습니다.

이 모든 API 호출은 비동기 작업이라는 점에 유의하세요; 이는 우리가 `asyncio`와 같은 비동기 라이브러리의 사용 없이도 실제로 코드에서 비동기 동작을 얻을 수 있음을 의미합니다!

In [17]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "방정식 '3x + 11 = 14'를 풀어줘"
)
thread2, run2 = create_thread_and_run("선형대수에 대해 설명해줘") # 동기적이라면 위에꺼가 완료되기 전까지 이 코드 작동 x
thread3, run3 = create_thread_and_run("나는 수학을 싫어해. 어떻게 하면 좋을까?")

# Now all Runs are executing...

모든 Run이 진행되고 나면, 각각을 기다린 후 응답을 받을 수 있습니다.

In [18]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1 # 비동기적이라 모든 요청들이 한번에 끝낼 수 있음
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "고마워")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

# Messages
user: 방정식 '3x + 11 = 14'를 풀어줘
assistant: x = 1.

# Messages
user: 선형대수에 대해 설명해줘
assistant: 선형대수는 벡터 공간과 선형 변환을 연구하는 수학의 한 분야입니다.

# Messages
user: 나는 수학을 싫어해. 어떻게 하면 좋을까?
assistant: 실용적인 목적이나 재미있는 방법으로 접근해보세요.

# Messages
user: 나는 수학을 싫어해. 어떻게 하면 좋을까?
assistant: 실용적인 목적이나 재미있는 방법으로 접근해보세요.
user: 고마워
assistant: 천만에!



끝났습니다!

여러분은 이 코드가 실제로는 우리의 수학 Assistant에 특화되어 있지 않다는 것을 눈치챘을 수도 있습니다... Assistant ID를 변경하기만 하면 어떤 새로운 Assistant에 대해서도 이 코드가 작동합니다! 이것이 Assistants API의 힘입니다.

## Tools

Assistants API의 핵심 기능 중 하나는 Code Interpreter, Retrieval 및 사용자 정의 Functions과 같은 도구를 우리의 Assistants에 장착할 수 있는 능력입니다. 각각에 대해 살펴보겠습니다.

### Code Interpreter

우리의 수학 과외 선생님에게 [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) 도구를 장착해보겠습니다. 이 작업은 대시보드에서 수행할 수 있습니다...


![Enabling code interpreter](images/assistants_overview_enable_code_interpreter.png)


...또는 Assistant ID를 사용하여 API를 통해 실행할 수 있습니다.

In [19]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_dL7GXryfiTzV7zAQ1rvE72xI',
 'created_at': 1728575057,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}],
 'response_format': {'type': 'text'},
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': []}, 'file_search': None},
 'top_p': 1.0}

이제 Assistant에게 새 도구를 사용하도록 요청해 보겠습니다.

In [20]:
thread, run = create_thread_and_run(
    "코드를 사용하여 처음 10개의 피보나치 숫자를 생성해줘"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: 코드를 사용하여 처음 10개의 피보나치 숫자를 생성해줘
assistant: 처음 10개의 피보나치 숫자는 [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]입니다.



그리고 끝났습니다! Assistant는 백그라운드에서 Code Interpreter를 사용했고, 최종 응답을 제공했습니다.

일부 사용 사례에서는 이 정도면 충분할 수 있습니다. 하지만 Assistant가 정확히 무엇을 하는지 더 자세히 알고 싶다면 Run의 Steps를 살펴볼 수 있습니다.

### Steps

Run은 하나 이상의 Steps로 구성됩니다. Run과 마찬가지로 각 Step에는 조회할 수 있는 `status`가 있습니다. 이는 사용자에게 Step의 진행 상황을 표시하는 데 유용합니다(예: Assistant가 코드를 작성하거나 검색을 수행하는 동안 스피너 표시).

In [21]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

각 Step의 `step_details`를 살펴보겠습니다.

In [24]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_l5e6dtjYHCBBayqxKcK2a6Oq',
   'code_interpreter': {'input': 'def fibonacci_sequence(n):\n    fib_sequence = [0, 1]\n    while len(fib_sequence) < n:\n        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])\n    return fib_sequence\n\nfirst_10_fibonacci = fibonacci_sequence(10)\nfirst_10_fibonacci',
    'outputs': []},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_vcwfkjbn2wTs5ZvfMBqQ6sB6'},
 'type': 'message_creation'}

null


두 Step의 `step_details`를 볼 수 있습니다:

1. `tool_calls` (하나의 Step에 하나 이상이 있을 수 있으므로 복수형)
2. `message_creation`

첫 번째 Step은 `tool_calls`로, 특히 `code_interpreter`를 사용하는데 여기에는 다음이 포함됩니다:

- 도구가 호출되기 전에 생성된 Python 코드인 `input`, 그리고
- Code Interpreter를 실행한 결과인 `output`.

두 번째 Step은 `message_creation`으로, 사용자에게 결과를 전달하기 위해 Thread에 추가된 `message`가 포함되어 있습니다.

### file search

Assistants API의 또 다른 강력한 도구는 [file search](https://platform.openai.com/docs/assistants/tools/file-search/quickstart)입니다: 질문에 답할 때 assistant가 지식 기반으로 사용할 파일을 업로드하는 기능입니다. 이 기능도 대시보드 또는 API에서 활성화할 수 있으며, 사용하고자 하는 파일을 업로드할 수 있습니다.

In [30]:
# Create a vector store
vector_store = client.beta.vector_stores.create(name="machine learning")
 
# Ready the files for upload to OpenAI
file_paths = ["data/language_models_are_unsupervised_multitask_learners.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [31]:
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)
show_json(assistant)

{'id': 'asst_kapTM3MdsjHAguqiTCXjX3Xy',
 'created_at': 1716706179,
 'description': None,
 'instructions': '너는 개인 수학 교사야. 질문에 한 문장 이하로 짧게 답해줘',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'file_search'}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': []},
  'file_search': {'vector_store_ids': ['vs_npS5ZqeENO4FeKrEGTepl6eJ']}},
 'top_p': 1.0}

In [32]:
thread, run = create_thread_and_run(
    "이 논문의 배경이 되는 수학 개념은 뭐야? 두 문장으로 설명해줘"
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: 이 논문의 배경이 되는 수학 개념은 뭐야? 두 문장으로 설명해줘
assistant: 이 논문의 배경이 되는 수학 개념은 확률적 모델링과 다중 작업 학습(multitask learning)입니다【4:0†source】.



> **Note**
> Retrieval에는 [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages)와 같은 더 복잡한 기능들이 있으며, 이는 다른 쿡북에서 다룰 수 있습니다.


In [ ]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

다음은 샘플 퀴즈가 어떻게 보일지에 대한 예입니다:

In [ ]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "이름이 뭐야?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "가장 좋아하는 색이 뭐야?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["빨간색", "파랑색", "초록색", "노란색"],
        },
    ],
)
print("Responses:", responses)

이제, Assistant가 호출할 수 있도록 이 함수의 인터페이스를 JSON 형식으로 정의해 보겠습니다:

In [ ]:
function_json = {
    "name": "display_quiz",
    "description": "학생에게 퀴즈를 표시하고 학생의 응답을 반환합니다. 하나의 퀴즈에 여러 개의 질문이 포함될 수 있습니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "제목과 옵션(객관식인 경우)이 있는 질문의 배열입니다.",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

다시 한번, 대시보드나 API를 통해 우리의 Assistant를 업데이트해 보겠습니다.

![Enabling custom function](images/assistants_overview_enable_function.png)

In [ ]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

이제 퀴즈를 요청해 봅시다.

In [ ]:
thread, run = create_thread_and_run(
    "두 가지 질문으로 퀴즈를 만들어줘. 하나는 주관식, 하나는 객관식으로, 그런 다음 답변에 대한 피드백을 보내줘"
)
run = wait_on_run(run, thread)
run.status

하지만 이제 Run의 `status`를 확인하면 `requires_action`이 표시됩니다. 좀 더 자세히 살펴보겠습니다.

In [ ]:
show_json(run)

`required_action` 필드는 도구가 우리에게 실행하고 그 결과를 Assistant에게 제출하도록 대기 중인 것을 나타냅니다. 구체적으로 `display_quiz` 함수입니다! 먼저 `name` 및 `arguments`를 구문 분석하는 것부터 시작해 보겠습니다.

> **참고**
> 이 경우에는 하나의 도구 호출하지만, 실제로는 Assistant는 여러 도구를 호출하기로 선택할 수 있습니다.

In [ ]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

이제 Assistant에서 제공한 인수를 사용하여 실제로 `display_quiz` 함수를 호출해 보겠습니다:

In [ ]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

좋습니다! (이 응답들은 이전에 모의로 만들었던 것입니다. 실제로는 이 함수 호출에서 입력을 다시 받아올 것입니다.)

이제 응답을 얻었으므로, Assistant에게 다시 제출해 보겠습니다. 이전에 분석한 `tool_call`에서 찾을 수 있는 `tool_call` ID가 필요합니다. 또한 응답들의 `list`를 `str`로 인코딩해야 합니다.

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

이제 다시 Run이 완료될 때까지 기다릴 수 있으며, Thread를 확인해 보겠습니다!

In [ ]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

완료했습니다! 🎉


## 결론
간략함을 유지하기 위해 다루지 않은 몇 가지 섹션도 있으므로 더 자세히 살펴볼 수 있는 몇 가지 리소스를 제공합니다:

- [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages): 파일 인용문 구문 분석
- [Files](https://platform.openai.com/docs/api-reference/assistants/file-object): Thread 범위 vs Assistant 범위
- [Parallel Function Calls](https://platform.openai.com/docs/guides/function-calling/parallel-function-calling): 하나의 단계에서 여러 도구 호출
- Multi-Assistant Thread Runs: 여러 Assistant에서 온 메시지가 있는 단일 Thread